## 2.1 내장 시퀀스 개요
파이썬 표준 라이브러리는 C로 구현된 다음과 같은 시퀀스형을 제공한다. 
- 컨테이너 시퀀스 : 서로 다른 자료형의 항목들을 담을 수 있는 ```list, tuple, collections, deque``` 형
- 균일 시퀀스 : 단 하나의 자료형만 담을 수 있는 ```str, bytes, bytearray, memoryview, array.array``` 형

시퀀스형은 가변성에 따라 분류할 수도 있다.
- 가변 시퀀스 : ```list, bytearray, array.array, collections, deque, memoryview``` 형
- 불변 시퀀스 : ```tuple, str, bytes``` 형

### 지능형 리스트와 가독성

In [1]:
symbols = '~!@#$%^'
codes = []

for symbol in symbols:
    codes.append(ord(symbol))
    
codes

[126, 33, 64, 35, 36, 37, 94]

In [2]:
codes = [ord(symbol) for symbol in symbols]
codes

[126, 33, 64, 35, 36, 37, 94]

---

In [3]:
# map() / filter() 비교
beyond_ascii = [ord(s) for s in symbols if ord(s) > 50]
beyond_ascii

[126, 64, 94]

In [4]:
beyond_ascii = list(filter(lambda c: c > 50, map(ord, symbols)))
beyond_ascii

[126, 64, 94]

In [5]:
import timeit

TIMES = 10000

SETUP = """
symbols = '$¢£¥€¤'
def non_ascii(c):
    return c > 127
"""

def clock(label, cmd):
    res = timeit.repeat(cmd, setup=SETUP, number=TIMES)
    print(label, *(f'{x:.3f}' for x in res))

clock('listcomp        :', '[ord(s) for s in symbols if ord(s) > 127]')
clock('listcomp + func :', '[ord(s) for s in symbols if non_ascii(ord(s))]')
clock('filter + lambda :', 'list(filter(lambda c: c > 127, map(ord, symbols)))')
clock('filter + func   :', 'list(filter(non_ascii, map(ord, symbols)))')

listcomp        : 0.018 0.009 0.009 0.009 0.007
listcomp + func : 0.011 0.009 0.008 0.007 0.006
filter + lambda : 0.005 0.005 0.005 0.005 0.005
filter + func   : 0.005 0.005 0.005 0.005 0.005


---

In [5]:
# 데카르트 곱
colors = ['black', 'white']
sizes = ['S', 'M', 'L']

tshirts = [(color, size) for color in colors for size in sizes]
tshirts

[('black', 'S'),
 ('black', 'M'),
 ('black', 'L'),
 ('white', 'S'),
 ('white', 'M'),
 ('white', 'L')]

---

In [6]:
# 제너레이터 표현식에서 튜플과 배열 초기화하기
tuple(ord(symbol) for symbol in symbols)

(126, 33, 64, 35, 36, 37, 94)

In [7]:
import array

array.array('I', (ord(symbol) for symbol in symbols))

array('I', [126, 33, 64, 35, 36, 37, 94])

---

In [8]:
for tshirt in ('%s %s' % (c, s) for c in colors for s in sizes):
    print(tshirt)

black S
black M
black L
white S
white M
white L


---

## 2.3 튜플은 단순한 불변 리스트가 아니다
튜플은 불변 리스트로 사용할 수도 있지만 필드명이 없는 레코드로 사용할 수도 있다. 

In [10]:
# 레코드로 사용된 튜플
lax_coordinates = (33.1234, -118.391241)
city, year, pop, chg, area = ('Seoul', 2003, 32488, 0.66, 8015)

traveler_ids = [('USA', '312141'), ('BRA', 'CE123135'), ('ESP', 'XDA208521')]

for passport in sorted(traveler_ids):
    print('%s/%s'% passport)

BRA/CE123135
ESP/XDA208521
USA/312141


In [11]:
# 튜플 언패킹
lax_coordinates = (33.9425, -118.15615)
latitude, longitude = lax_coordinates
latitude

33.9425

In [12]:
longitude

-118.15615

In [13]:
t = (20, 8)
divmod(*t)

quotient, remainder = divmod(*t)
quotient, remainder

(2, 4)

In [15]:
a, b, *rest = range(4)
a, b, rest

(0, 1, [2, 3])

In [7]:
a, *body, c, d, = range(5)
a, body, c, d

(0, [1, 2], 3, 4)

---

In [17]:
# 명명된 튜플
from collections import namedtuple

City = namedtuple('City', 'name country population coordinates')
seoul = City('Seoul', 'KO', 42.695, (42.156112, 138.156178))
seoul

City(name='Seoul', country='KO', population=42.695, coordinates=(42.156112, 138.156178))

In [18]:
seoul.population

42.695

In [19]:
seoul.coordinates

(42.156112, 138.156178)

In [20]:
seoul[1]

'KO'

In [29]:
seoul = City._make(seoul)

In [30]:
seoul._asdict()

{'name': 'Seoul',
 'country': 'KO',
 'population': 42.695,
 'coordinates': (42.156112, 138.156178)}

In [31]:
for key, val in seoul._asdict().items():
    print(key, val)

name Seoul
country KO
population 42.695
coordinates (42.156112, 138.156178)


---

## 2.4 슬라이싱

In [32]:
l = list(range(10))
l[2:5] = [20, 35]
l

[0, 1, 20, 35, 5, 6, 7, 8, 9]

In [33]:
del l[5:7]
l

[0, 1, 20, 35, 5, 8, 9]

---

## 2.5 시퀀스에 덧셈과 곱셈 연산자 사용하기

In [8]:
l = [1, 2, 3]
l * 5

[1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3]

In [9]:
5 * 'abcd'

'abcdabcdabcdabcdabcd'

In [11]:
board = [['_'] * 3 for i in range(3)]
board

[['_', '_', '_'], ['_', '_', '_'], ['_', '_', '_']]

---

## 2.6 시퀀스의 복합 할당

In [14]:
l = [1, 2, 3]
id(l)

4425554112

In [15]:
l *= 2
l

[1, 2, 3, 1, 2, 3]

In [16]:
id(l)

4425554112

In [17]:
t = (1, 2, 3)
id(t)

4569994624

In [18]:
t *= 2
id(t)

4571573984

In [19]:
t = (1, 2, [30, 40])
t[2] += [50, 60]

TypeError: 'tuple' object does not support item assignment

In [20]:
t

(1, 2, [30, 40, 50, 60])

In [21]:
import dis

dis.dis('s[a] += b')

  1           0 LOAD_NAME                0 (s)
              2 LOAD_NAME                1 (a)
              4 DUP_TOP_TWO
              6 BINARY_SUBSCR
              8 LOAD_NAME                2 (b)
             10 INPLACE_ADD
             12 ROT_THREE
             14 STORE_SUBSCR
             16 LOAD_CONST               0 (None)
             18 RETURN_VALUE


---

## 2.7 list.sort()와 sorted() 내장 함수

In [22]:
fruits = ['grape', 'raspberry', 'apple', 'banana']
sorted(fruits)

['apple', 'banana', 'grape', 'raspberry']

In [23]:
fruits

['grape', 'raspberry', 'apple', 'banana']

In [24]:
sorted(fruits, reverse=True)

['raspberry', 'grape', 'banana', 'apple']

In [25]:
sorted(fruits, key=len)

['grape', 'apple', 'banana', 'raspberry']

In [32]:
sorted(fruits, key=str.lower)

['apple', 'banana', 'grape', 'raspberry']

In [26]:
fruits.sort()

In [27]:
fruits

['apple', 'banana', 'grape', 'raspberry']

---

## 2.8 정렬된 시퀀스를 bisect로 관리하기

In [36]:
import bisect
import sys

HAYSTACK = [1, 4, 5, 6, 8, 12, 15, 20, 21, 23, 23, 26, 29, 30]
NEEDLES = [0, 1, 2, 5, 8, 10, 22, 23, 29, 30, 31]

ROW_FMT = '{0:2d} @ {1:2d}    {2}{0:<2d}'

def demo(bisect_fn):
    for needle in reversed(NEEDLES):
        position = bisect_fn(HAYSTACK, needle)  # <1>
        offset = position * '  |'  # <2>
        print(ROW_FMT.format(needle, position, offset))  # <3>

if __name__ == '__main__':

    if sys.argv[-1] == 'left':    # <4>
        bisect_fn = bisect.bisect_left
    else:
        bisect_fn = bisect.bisect

    print('DEMO:', bisect_fn.__name__)  # <5>
    print('haystack ->', ' '.join(f'{n:2}' for n in HAYSTACK))
    demo(bisect_fn)

DEMO: bisect_right
haystack ->  1  4  5  6  8 12 15 20 21 23 23 26 29 30
31 @ 14      |  |  |  |  |  |  |  |  |  |  |  |  |  |31
30 @ 14      |  |  |  |  |  |  |  |  |  |  |  |  |  |30
29 @ 13      |  |  |  |  |  |  |  |  |  |  |  |  |29
23 @ 11      |  |  |  |  |  |  |  |  |  |  |23
22 @  9      |  |  |  |  |  |  |  |  |22
10 @  5      |  |  |  |  |10
 8 @  5      |  |  |  |  |8 
 5 @  3      |  |  |5 
 2 @  1      |2 
 1 @  1      |1 
 0 @  0    0 


In [37]:
import bisect
import random

SIZE = 7

random.seed(1729)

my_list = []
for i in range(SIZE):
    new_item = random.randrange(SIZE * 2)
    bisect.insort(my_list, new_item)
    print(f'{new_item:2d} -> {my_list}')

10 -> [10]
 0 -> [0, 10]
 6 -> [0, 6, 10]
 8 -> [0, 6, 8, 10]
 7 -> [0, 6, 7, 8, 10]
 2 -> [0, 2, 6, 7, 8, 10]
10 -> [0, 2, 6, 7, 8, 10, 10]


---

## 2.9 리스트가 답이 아닐 때

In [38]:
from array import array
from random import random, seed

floats = array('d', (random() for i in range(10 ** 7)))
floats[-1]

0.5963321947530882

In [39]:
import array
numbers = array.array('h', [-2, -1, 0, 1, 2])
memv = memoryview(numbers)
len(memv)

5

In [40]:
memv[0]

-2

In [41]:
memv_oct = memv.cast('B')
memv_oct.tolist()

[254, 255, 255, 255, 0, 0, 1, 0, 2, 0]

In [42]:
memv_oct[5] = 4
numbers

array('h', [-2, -1, 1024, 1, 2])

In [43]:
# 덱
from collections import deque
dq = deque(range(10), maxlen=10)
dq

deque([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [44]:
dq.rotate(3)
dq

deque([7, 8, 9, 0, 1, 2, 3, 4, 5, 6])

In [45]:
dq.rotate(-4)
dq

deque([1, 2, 3, 4, 5, 6, 7, 8, 9, 0])

In [46]:
dq.appendleft(-1)
dq

deque([-1, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [47]:
dq.extend([11, 22, 33])
dq

deque([3, 4, 5, 6, 7, 8, 9, 11, 22, 33])

In [48]:
dq.extendleft([10, 20, 30, 40])
dq

deque([40, 30, 20, 10, 3, 4, 5, 6, 7, 8])